# Training de Modelo 1.1. – Clasificador de CV

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow import keras

2025-12-02 19:16:00.378358: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Importación del df

In [ ]:
import pandas as pd

df = pd.read_csv('/Users/alexmiliandieguez/code/TechCareer/data/model1_cv_role/3.processed/v1_english/cv_labeled_final.csv')
df.head()

,cv_id,cv_text,role_label_final,role_raw,source_dataset,resume_len
0,1,Python Developer Python Developer Python Devel...,python_developer,Python_Developer,dataset1_avishek,3467
1,2,R&D Engineer R&D Engineer R&D Engineer - Nokia...,python_developer,Python_Developer,dataset1_avishek,2812
2,3,Sr. Full Stack Developer Sr. Full Stack Develo...,python_developer,Python_Developer,dataset1_avishek,16606
3,4,Sr. Full Stack Python Developer Sr. Full Stack...,python_developer,Python_Developer,dataset1_avishek,9253
4,5,Sr. Python Developer Sr. Python Developer Sr. ...,python_developer,Python_Developer,dataset1_avishek,15659


Quitamos columnas que no necesitamos

In [ ]:
df = df.loc[:, ['cv_text', 'role_label_final']]

df.rename(columns={'cv_text': 'cv', 'role_label_final': 'role'}, inplace=True)
df.head()


,cv,role
0,Python Developer Python Developer Python Devel...,python_developer
1,R&D Engineer R&D Engineer R&D Engineer - Nokia...,python_developer
2,Sr. Full Stack Developer Sr. Full Stack Develo...,python_developer
3,Sr. Full Stack Python Developer Sr. Full Stack...,python_developer
4,Sr. Python Developer Sr. Python Developer Sr. ...,python_developer


Separación de etiquetas + train test

In [ ]:
from sklearn.model_selection import train_test_split

X_texts = df['cv'].values 
y_labels = df['role'].values 

X_train_texts, X_test_texts, y_train_labels, y_test_labels = train_test_split(
    X_texts, y_labels, test_size=0.2, random_state=42)


Vectorizar CV (máx 5000 carácteres distintos)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(X_train_texts).toarray()
X_test = vectorizer.transform(X_test_texts).toarray()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
y_train = le.fit_transform(y_train_labels)
y_test = le.transform(y_test_labels)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = y_train.shape[1]


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))


from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

model.summary()


/Users/alexmiliandieguez/code/TechCareer/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │     1,280,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │         2,064 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,315,216 (5.02 MB)

 Trainable params: 1,315,216 (5.02 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=20,
                    validation_data=(X_test, y_test),
                    verbose=2)


Epoch 1/20
207/207 - 4s - 19ms/step - accuracy: 0.6978 - loss: 1.0194 - val_accuracy: 0.8367 - val_loss: 0.4584
Epoch 2/20
207/207 - 2s - 11ms/step - accuracy: 0.8649 - loss: 0.3768 - val_accuracy: 0.8736 - val_loss: 0.3638
Epoch 3/20
207/207 - 2s - 11ms/step - accuracy: 0.9351 - loss: 0.2128 - val_accuracy: 0.8826 - val_loss: 0.3306
Epoch 4/20
207/207 - 2s - 11ms/step - accuracy: 0.9614 - loss: 0.1314 - val_accuracy: 0.8772 - val_loss: 0.3571
Epoch 5/20
207/207 - 2s - 11ms/step - accuracy: 0.9814 - loss: 0.0714 - val_accuracy: 0.8947 - val_loss: 0.3519
Epoch 6/20
207/207 - 2s - 11ms/step - accuracy: 0.9912 - loss: 0.0401 - val_accuracy: 0.8923 - val_loss: 0.3776
Epoch 7/20
207/207 - 2s - 11ms/step - accuracy: 0.9956 - loss: 0.0241 - val_accuracy: 0.8923 - val_loss: 0.4211
Epoch 8/20
207/207 - 2s - 11ms/step - accuracy: 0.9974 - loss: 0.0143 - val_accuracy: 0.8917 - val_loss: 0.4469
Epoch 9/20
207/207 - 2s - 11ms/step - accuracy: 0.9989 - loss: 0.0097 - val_accuracy: 0.8826 - val_loss:

Tenemos overfitting. Hay que afrontar el entrenamiento de otra manera.